<a href="https://colab.research.google.com/github/harsh-sengar/tuberculosis-drug-resistance-classification/blob/master/tuberculosisMachineLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import pandas as pd
import numpy as np


In [ ]:
#drive.mount('/content/drive')
#path = "/content/drive/My Drive/IIT Kanpur/Genomics/Assignment1/data/X_trainData_1.csv"
#X_trainData_1 = pd.read_csv(path)
#path1 = "/content/drive/My Drive/IIT Kanpur/Genomics/Assignment1/data/Y_trainData_1.csv"
#Y_trainData_1 = pd.read_csv(path1)

In [38]:
dataset = pd.read_csv('https://raw.githubusercontent.com/harsh-sengar/tuberculosis-drug-resistance-classification/master/SMOTE/SMOTE_updated_X_trainData_column_modified_INH.csv?token=AHJLH6DZRGY7SXXJH2NNKKC7P3X2K')
dataset = np.array(dataset)
X_trainData_1 = (dataset[:,:219])
Y_trainData_1 = (dataset[:,219])

In [39]:
x_train1 = np.array(X_trainData_1)
y_train1 = np.array(Y_trainData_1)

In [7]:
test = pd.read_csv('https://raw.githubusercontent.com/harsh-sengar/tuberculosis-drug-resistance-classification/master/SMOTE/X_testData_1_modified.csv?token=AHJLH6CGQ6PAAI5LGW6R4JS7P3VNC')
x_test = np.array(test)


(1000, 219)

# Xgboost classifier

In [40]:
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score

clf = XGBClassifier(max_depth=9, eta = 0.4, gamma = 3, nrounds = 100)

clf.fit(x_train1, y_train1)
y_pred = clf.predict_proba(x_test)

In [41]:
y_pred = y_pred[:,1]

In [42]:
submission_data = pd.read_csv('https://raw.githubusercontent.com/harsh-sengar/tuberculosis-drug-resistance-classification/master/SMOTE/Y_testData_1_nolabels_INH.csv?token=AHJLH6B2T7OSJTTZNRXWYHK7P3X6G')

ids=submission_data.iloc[:,:1].values
result=list()

for i in ids:
    result.append([ i[0], y_pred[i[0]-1] ])

result.insert(0, ['ID','INH'])
print(result)

[['ID', 'INH'], [1, 0.010637029], [2, 0.008616114], [3, 0.008898048], [4, 0.008616114], [5, 0.010755935], [6, 0.008898048], [7, 0.010755935], [8, 0.008589537], [9, 0.5885498], [10, 0.008589537], [11, 0.010383682], [12, 0.008616114], [13, 0.008616114], [14, 0.7561494], [15, 0.008317296], [16, 0.010755935], [17, 0.8358521], [18, 0.008616114], [19, 0.010383682], [20, 0.966459], [21, 0.010383682], [22, 0.008616114], [23, 0.009183669], [24, 0.5885498], [25, 0.010755935], [26, 0.0099763945], [27, 0.011096162], [28, 0.011463084], [29, 0.8907403], [30, 0.010755935], [31, 0.008317296], [32, 0.3293941], [33, 0.008616114], [34, 0.010383682], [35, 0.0075301696], [36, 0.8907403], [37, 0.010755935], [38, 0.010383682], [39, 0.010755935], [40, 0.5885498], [41, 0.008898048], [42, 0.008339551], [43, 0.008898048], [44, 0.9753091], [45, 0.010755935], [46, 0.9551493], [47, 0.010383682], [48, 0.008616114], [49, 0.008616114], [50, 0.008317296], [51, 0.9498514], [52, 0.008317296], [53, 0.5176222], [54, 0.0072

In [43]:
import csv
file = open('INH_result.csv', 'w+', newline ='') 
  
# writing the data into the file 
with file:     
    write = csv.writer(file) 
    write.writerows(result) 


In [44]:
from google.colab import files
files.download("INH_result.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Neural Network for Single Task Classification

In [33]:
import pandas
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

X = x_train1
Y = y_train1

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
encoded_Y = np_utils.to_categorical(encoded_Y)

def neuralNetworkModelWithTHREEoutputs():
    model = Sequential()
    model.add(Dense(200, input_dim = 222, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(150, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(75, activation='relu'))
    model.add(Dense(15, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=neuralNetworkModelWithTHREEoutputs, epochs=20, batch_size=50, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("neuralNetwork: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))



neuralNetwork: 96.46% (0.71%)
